In [104]:
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


In [105]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import warnings; warnings.simplefilter('ignore')

def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def filter_keywords(x, s):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

cols = ['id', 'movieId', 'title', 'genres', 'description', 'keywords', \
        'cast', 'director', 'spoken_languages', 'production_companies',\
        'production_countries', 'popularity', 'year', 'vote_average',\
         'vote_count', 'wr']

def read_dataset(metadata_path, links_small_path, credits_path,keywords_path):
    meta = pd.read_csv(metadata_path)
    links_small = pd.read_csv(links_small_path)
    credits = pd.read_csv(credits_path)
    keywords = pd.read_csv(keywords_path)
    return meta, links_small, credits, keywords

def cal_weighted_rating(meta, percentile=0.95):
    vote_counts = meta[meta['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = meta[meta['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    meta = meta[(meta['vote_count'].notnull()) \
                     & (meta['vote_average'].notnull())]
    meta['vote_count'] = meta['vote_count'].astype('int')
    meta['vote_average'] = meta['vote_average'].astype('int')
    meta['wr'] = meta.apply(lambda x: weighted_rating(x, m, C), axis=1)

    return meta
    
def movie_feature(metadata_path, links_small_path, credits_path,keywords_path, \
                  percentile=0.95, more_weight_on = None, \
                    stemmer = SnowballStemmer('english'), cols=cols):
    # read dataset
    meta, links_small, credits, keywords = read_dataset(metadata_path, \
        links_small_path, credits_path,keywords_path)

    # change type + drop
    links_small = links_small[links_small['tmdbId'].notnull()]
    links_small['tmdbId'] = links_small['tmdbId'].astype('int')

    meta = meta.drop([19730, 29503, 35587])
    meta['popularity'] = meta[meta['popularity'].notnull()]['popularity'].astype('float')
    meta['id'] = meta['id'].astype('int')
    meta['year'] = pd.to_datetime(meta['release_date'], errors='coerce').apply(\
        lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
    meta = meta[meta.production_companies.notnull()]
    keywords['id'] = keywords['id'].astype('int')
    credits['id'] = credits['id'].astype('int')

    # calcualte weighted rating for movies
    meta = cal_weighted_rating(meta)

    # merge meta + link small => create a smaller dataset for recommend
    smd = meta[meta['id'].isin(links_small['tmdbId'])]

    # create description feature
    smd['tagline'] = smd['tagline'].fillna('')
    smd['description'] = smd['overview'] + smd['tagline']
    smd['description'] = smd['description'].fillna('')
    
    # merge credit + keywords + links_small
    smd = smd.merge(credits, on='id')
    smd = smd.merge(keywords, on='id')
    smd = smd.merge(links_small, left_on='id', right_on='tmdbId')

    # feature engineering
    literal_features = ['cast', 'spoken_languages', 'genres', 'keywords',\
                        'production_companies', 'production_countries']
    for fearture in literal_features:
        smd[fearture] = smd[fearture].apply(literal_eval)
        smd[fearture] = smd[fearture].apply(lambda x: [i['name'] \
                                            for i in x] if isinstance(x, list) else [])
    smd['crew'] = smd['crew'].apply(literal_eval)
    smd['director'] = smd['crew'].apply(get_director)

    # top 3 actors
    smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
   
    # Strip Spaces and Convert to Lowercase 
    smd['cast'] = smd['cast'].apply(lambda x: [str.lower(\
        i.replace(" ", "")) for i in x])
    smd['director'] = smd['director'].astype('str').apply(\
        lambda x: str.lower(x.replace(" ", "")))
    if more_weight_on:
        smd[more_weight_on] = smd[more_weight_on].apply(lambda x: [x,x,x])
    
    # choose keywords appear more than once + stemming
    s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
    s = s.value_counts()
    s = s[s > 1]
    smd['keywords'] = smd['keywords'].apply(lambda x: filter_keywords(x, s))
    smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
    smd['keywords'] = smd['keywords'].apply(lambda x: \
                                            [str.lower(i.replace(" ", "")) for i in x])

    return smd[cols]
    


In [106]:
credits_ = r'/kaggle/input/the-movies-dataset/credits.csv'
keywords = r'/kaggle/input/the-movies-dataset/keywords.csv'
links = r'/kaggle/input/the-movies-dataset/links_small.csv'
movies_metadata = r'/kaggle/input/the-movies-dataset/movies_metadata.csv'
smd = movie_feature(movies_metadata, links, credits_, keywords, more_weight_on='director')
# print(smd.columns)

In [108]:
smd

,id,movieId,title,genres,description,keywords,cast,director,spoken_languages,production_companies,production_countries,popularity,year,vote_average,vote_count,wr
0,862,1,Toy Story,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...","[jealousi, toy, boy, friendship, friend, rival...","[tomhanks, timallen, donrickles]","[johnlasseter, johnlasseter, johnlasseter]",[English],[Pixar Animation Studios],[United States of America],21.946943,1995,7,5415,6.869770
1,8844,2,Jumanji,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,"[boardgam, disappear, basedonchildren'sbook, n...","[robinwilliams, jonathanhyde, kirstendunst]","[joejohnston, joejohnston, joejohnston]","[English, Français]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],17.015539,1995,6,2413,5.884891
2,15602,3,Grumpier Old Men,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,"[fish, bestfriend, duringcreditssting]","[waltermatthau, jacklemmon, ann-margret]","[howarddeutch, howarddeutch, howarddeutch]",[English],"[Warner Bros., Lancaster Gate]",[United States of America],11.712900,1995,6,92,5.376968
3,31357,4,Waiting to Exhale,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...","[basedonnovel, interracialrelationship, single...","[whitneyhouston, angelabassett, lorettadevine]","[forestwhitaker, forestwhitaker, forestwhitaker]",[English],[Twentieth Century Fox Film Corporation],[United States of America],3.859495,1995,6,34,5.299755
4,11862,5,Father of the Bride Part II,[Comedy],Just when George Banks has recovered from his ...,"[babi, midlifecrisi, confid, age, daughter, mo...","[stevemartin, dianekeaton, martinshort]","[charlesshyer, charlesshyer, charlesshyer]",[English],"[Sandollar Productions, Touchstone Pictures]",[United States of America],8.387519,1995,5,173,5.175099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,[Drama],A man must cope with the loss of his wife and ...,[friendship],"[sidneypoitier, wendycrewson, jayo.sanders]","[greggchampion, greggchampion, greggchampion]",[],[Nasser Entertainment],[United States of America],0.038998,2001,7,1,5.248931
9215,392572,162542,Rustom,"[Thriller, Romance]","Rustom Pavri, an honourable officer of the Ind...",[bollywood],"[akshaykumar, ileanad'cruz, eshagupta]","[tinusureshdesai, tinusureshdesai, tinusureshd...",[हिन्दी],[KriArj Entertainment],[India],7.333139,2016,7,25,5.340490
9216,402672,162672,Mohenjo Daro,"[Adventure, Drama, History, Romance]","Village lad Sarman is drawn to big, bad Mohenj...",[bollywood],"[hrithikroshan, poojahegde, kabirbedi]","[ashutoshgowariker, ashutoshgowariker, ashutos...",[हिन्दी],"[UTV Motion Pictures, Ashutosh Gowariker Produ...",[India],1.423358,2016,6,26,5.287576
9217,315011,163056,Shin Godzilla,"[Action, Adventure, Drama, Horror, Science Fic...",From the mind behind Evangelion comes a hit la...,"[monster, godzilla, giantmonst, destruct, kaiju]","[hirokihasegawa, yutakatakenouchi, satomiishih...","[hideakianno, hideakianno, hideakianno]","[Italiano, Deutsch, English, 日本語]","[Cine Bazar, Toho Pictures]",[Japan],9.285519,2016,6,152,5.440760


In [109]:
smd["description"][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [110]:
smd = smd[['id', 'movieId', 'title', 'description']]
smd = smd.drop_duplicates()

In [111]:
smd

,id,movieId,title,description
0,862,1,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,2,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,3,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,4,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,5,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...
9214,159550,161944,The Last Brickmaker in America,A man must cope with the loss of his wife and ...
9215,392572,162542,Rustom,"Rustom Pavri, an honourable officer of the Ind..."
9216,402672,162672,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj..."
9217,315011,163056,Shin Godzilla,From the mind behind Evangelion comes a hit la...


### TF-IDF

In [112]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])
tfidf_matrix

<9082x268124 sparse matrix of type '<class 'numpy.float64'>'
	with 539409 stored elements in Compressed Sparse Row format>

In [113]:
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [114]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [115]:
cosine_sim.shape

(9082, 9082)

In [116]:
smd = smd.reset_index()
smd = smd.drop(columns='index')
movie_ids = smd['movieId']
indices = pd.Series(smd.index, index=smd['movieId'])

In [117]:
train_df = pd.read_csv('/kaggle/input/train-set-from-ratings/train_set.csv')

In [118]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [119]:
def get_top_n_similar(recommended, movie, cosine_sim, n_rec = 10):
    try:
        idx = indices[movie]
    except:
        # In case movie is not available in cosine_sim (or not in metadata)
        pass
    else:
        # top 10 most similar
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:n_rec+1]

        for movie_rec in sim_scores:
            # change index -> movieId
            movie_recommended = (movie_ids.iloc[movie_rec[0]], movie_rec[1])
            recommended.append(movie_recommended)
    return recommended

In [120]:
def get_recommendations(user_id, cosine_sim, n_rec = 10):
    movies_id = train_df[train_df.userId == user_id].movieId 
    recommended = []
    for movie in movies_id:
        recommended = get_top_n_similar(recommended, movie, cosine_sim, n_rec = 10)
            
    # create a DataFrame to store all movies recommended            
    recommended = pd.DataFrame(recommended)  
    recommended.columns = ['movieId', 'sim']
    recommended = recommended.sort_values('sim')[::-1].head(n_rec)
    recommended['userId'] = user_id
    return recommended.drop(columns='sim')

In [121]:
def full_recommend(df, cosine_sim):
    rec_df = pd.DataFrame()
    for user in df.userId.value_counts().index:
        rec_df = pd.concat([rec_df, get_recommendations(user, cosine_sim)])
    return rec_df

### Example

In [122]:
train_df

,userId,movieId,rating,timestamp
0,431,2863,4.0,1165548515
1,571,7173,2.0,1334343358
2,77,223,4.5,1163004353
3,580,1032,4.0,1165291033
4,624,1221,5.0,1019124147
...,...,...,...,...
74998,547,5810,3.0,1415444349
74999,418,1835,4.0,1132180632
75000,5,33679,4.0,1163374517
75001,358,905,5.0,957479957


In [123]:
smd[smd.title == 'The Dark Knight']

,id,movieId,title,description
6897,155,58559,The Dark Knight,Batman raises the stakes in his war on crime. ...


In [124]:
recommend_for_film = get_top_n_similar([], 58559, cosine_sim)
recommend_for_film

[(91529, 0.17134442806421551),
 (153, 0.12242142359950756),
 (1377, 0.10087530623328408),
 (99813, 0.08475042039923282),
 (79274, 0.08418256868195052),
 (592, 0.08161367173274224),
 (90603, 0.07779324829084786),
 (3213, 0.06961440545074826),
 (3386, 0.06174734075607319),
 (98124, 0.06092931350123673)]

In [125]:
smd[smd["movieId"].isin(pd.DataFrame(get_top_n_similar([], 58559, cosine_sim))[0])]

,id,movieId,title,description
132,414,153,Batman Forever,The Dark Knight of Gotham City confronts a das...
524,268,592,Batman,The Dark Knight of Gotham City begins his war ...
1113,364,1377,Batman Returns,"Having defeated the Joker, Batman now faces th..."
2578,14919,3213,Batman: Mask of the Phantasm,An old flame of Bruce Wayne's strolls into tow...
2695,820,3386,JFK,New Orleans District Attorney Jim Garrison dis...
7561,40662,79274,Batman: Under the Red Hood,Batman faces his ultimate challenge as the mys...
7896,69735,90603,Batman: Year One,Two men come to Gotham City: Bruce Wayne after...
7926,49026,91529,The Dark Knight Rises,Following the death of District Attorney Harve...
8160,123025,98124,"Batman: The Dark Knight Returns, Part 1",Batman has not been seen for ten years. A new ...
8222,142061,99813,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...


### Count Vectorize

In [126]:
count_vec = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_vec_matrix = count_vec.fit_transform(smd['description'])
count_vec_matrix

<9082x268124 sparse matrix of type '<class 'numpy.int64'>'
	with 539409 stored elements in Compressed Sparse Row format>

In [127]:
count_cosine_sim = cosine_similarity(count_vec_matrix, count_vec_matrix)
count_cosine_sim

array([[1.        , 0.01206221, 0.        , ..., 0.        , 0.        ,
        0.01031366],
       [0.01206221, 1.        , 0.02509626, ..., 0.        , 0.01123879,
        0.01965608],
       [0.        , 0.02509626, 1.        , ..., 0.01451717, 0.01226925,
        0.        ],
       ...,
       [0.        , 0.        , 0.01451717, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.01123879, 0.01226925, ..., 0.        , 1.        ,
        0.00960961],
       [0.01031366, 0.01965608, 0.        , ..., 0.        , 0.00960961,
        1.        ]])

In [128]:
smd[smd["movieId"].isin(pd.DataFrame(get_top_n_similar([], 58559, count_cosine_sim))[0])]

,id,movieId,title,description
132,414,153,Batman Forever,The Dark Knight of Gotham City confronts a das...
524,268,592,Batman,The Dark Knight of Gotham City begins his war ...
1113,364,1377,Batman Returns,"Having defeated the Joker, Batman now faces th..."
2578,14919,3213,Batman: Mask of the Phantasm,An old flame of Bruce Wayne's strolls into tow...
4488,31598,6180,Q & A,A young district attorney seeking to prove a c...
7561,40662,79274,Batman: Under the Red Hood,Batman faces his ultimate challenge as the mys...
7896,69735,90603,Batman: Year One,Two men come to Gotham City: Bruce Wayne after...
7926,49026,91529,The Dark Knight Rises,Following the death of District Attorney Harve...
8160,123025,98124,"Batman: The Dark Knight Returns, Part 1",Batman has not been seen for ten years. A new ...
8222,142061,99813,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...
